In [ ]:
import csv
from glob import glob
import json
import os

In [ ]:
ERR_NAME_MAPPING = {
    "trans_err": "mATE",
    "scale_err": "mASE",
    "orient_err": "mAOE",
}

CLASSES = (
    "CAR",
    "TRAILER",
    "TRUCK",
    "VAN",
    "PEDESTRIAN",
    "BUS",
    "MOTORCYCLE",
    "BICYCLE",
    "EMERGENCY_VEHICLE",
    "OTHER",
)

HEADERS = [
    "id",
    "eval_type",
    "mAP",
    "mAOE",
    "mATE",
    "mASE",
    "num_gt_bboxes"
]

for x in CLASSES:
    HEADERS.append(f"num_{x}")

for m in ["mAP", "mAOE", "mATE", "mASE"]:
    for x in CLASSES:
        HEADERS.append(f"{x}_{m}")


In [ ]:
# search_path = '../checkpoints/80-20/*'
# out_filename = '80s20-summary.csv'

search_path = '../checkpoints/80-10-10/*'
csv_out_path = '../results/80s10s10-summary.csv'

runs_paths = []

for x in glob(search_path):
    # check if test_results.json exists inside the folder
    if len(glob(x + '/test_results.json')) == 0:
        continue
    runs_paths.append(x + '/test_results.json')

In [ ]:
print("Will compile results for: ")
for i, x in enumerate(runs_paths):
    print(f"{i:<5}", x)

# csv file creation

In [ ]:
with open(csv_out_path, "w") as f:
    writer = csv.DictWriter(f, fieldnames=HEADERS)
    writer.writeheader()
    for path in runs_paths:
        id = os.path.basename(os.path.dirname(path))
        data = None
        with open(path, "rb") as json_file:
            data = json.load(json_file)

        for eval_type, eval_data in data.items():
            row = {
                "id": id,
                "eval_type": eval_type,
                "mAP": eval_data["mean_ap"],
                "num_gt_bboxes": eval_data["total_gt_bboxes"],
            }

            for x in CLASSES:
                if x in eval_data["class_counts"]:
                    row.update({f"num_{x}": eval_data["class_counts"][x]})
                else:
                    row.update({f"num_{x}": ""})

            for tp_name, tp_val in eval_data["tp_errors"].items():
                if tp_name in ERR_NAME_MAPPING:
                    row.update({ERR_NAME_MAPPING[tp_name]: tp_val})

            class_aps = eval_data["mean_dist_aps"]
            class_tps = eval_data["label_tp_errors"]

            for class_name in class_aps.keys():
                row.update({f"{class_name}_mAP": class_aps[class_name]})
                for tp_name, tp_val in class_tps[class_name].items():
                    if tp_name in ERR_NAME_MAPPING:
                        row.update(
                            {
                                f"{class_name}_{ERR_NAME_MAPPING[tp_name]}": class_tps[class_name][
                                    tp_name
                                ]
                            }
                        )

            writer.writerows([row])